In [9]:
%matplotlib notebook
from ipywidgets import *
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import os.path
import re
import json
import numpy as np
from numpy import genfromtxt
import pandas as pd
from keras.layers import Dense, Input, average
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import mean_absolute_error
from keras.layers import Dropout, Flatten,Conv1D
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from keras.models import load_model
import tensorflow as tf
from pmdarima.arima import auto_arima
from sklearn import preprocessing
from rpy2.robjects.packages import STAP
import math
import uber
import uber2
import time
tf.compat.v1.logging.set_verbosity(tf.logging.ERROR)
import rpy2
from rpy2.rinterface_lib.callbacks import logger as rpy2_logger
import logging
rpy2_logger.setLevel(logging.ERROR)   # will display errors, but not warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [10]:
import keras
print(keras.__version__)
print(tf.__version__)

2.2.4
1.14.0


In [11]:
DATASET_STATES_CSV = 'States_Timeseries.csv'
FOUR_THETA_FILE = 'four_theta.r'
THETA_FILE = 'theta.r'

FORECASTING_STEPS = 6
RANDOM_SEED = 3

SCALER_STANDARD = preprocessing.StandardScaler
SCALER_MINMAX = preprocessing.MinMaxScaler
DATA_SCALER = SCALER_MINMAX


In [12]:
def normalize_df(df,scaler):
    x = df.values #returns a numpy array
    x_train = df.values[:-6]
    display(x_train.shape)
    scaler = scaler.fit(x_train)
    x_scaled = scaler.transform(x)
    df = pd.DataFrame(x_scaled, columns=df.columns, index=df.index)
    return df

def normalizeSeries(df,scaler):
    x = np.array(df.values)
    x = np.reshape(x,(-1,1))
    x = scaler.fit_transform(x)
    x = x[:,0]
    return x

def normalizeInputTouple(x,y,scaler):
    x = np.array(x)
    x = np.reshape(x,(-1,1))
    x = scaler.fit_transform(x)
    x = x[:,0]
    y = y.reshape(-1,1)
    y = scaler.transform(y)
    y = y[:,0]
    return x,y

def getTrainTestSplit(series):
    series_np = np.array(series)
    return series_np[:-FORECASTING_STEPS],series_np[-FORECASTING_STEPS:]

def reshapeInputs(data):
    data = np.array(data)
    return np.reshape(data, (data.shape[0], data.shape[1],1))

def create_dataset(dataset,look_back):
    dataX, dataY = [], []
    for i in range(0,len(dataset) - look_back -FORECASTING_STEPS+1, int(look_back/2)):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back:i+look_back+FORECASTING_STEPS])
    return reshapeInputs(np.array(dataX)), np.array(dataY)

def create_dataset_scaled(dataset,scaler,LOOK_BACK):
    dataX, dataY = [], []
    for i in range(0,len(dataset) - LOOK_BACK -FORECASTING_STEPS+1, int(LOOK_BACK/2)):
        x = dataset[i:(i + LOOK_BACK)]
        y = dataset[i + LOOK_BACK:i+LOOK_BACK+FORECASTING_STEPS]
        normalizeInputTouple(x,y,scaler)
        dataX.append(x)
        dataY.append(y)
    return reshapeInputs(dataX), np.array(dataY)

def getBaselineLSTM():
    model = Sequential()
    model.add(LSTM(16, input_shape=(LOOK_BACK, 1),return_sequences=True))
    model.add(LSTM(8))
    model.add(Dense(FORECASTING_STEPS, activation='sigmoid'))
    model.compile(loss='mse', optimizer='adam', metrics=['mean_absolute_error'])
    return model


def evaluateModel(model,x,y,scaler,wsc = False):
    prediction = model.predict(x)
    if not scaler == None:
        prediction = scaler.inverse_transform(prediction)
        y = scaler.inverse_transform(y)
    score = mean_squared_error(y.ravel(),prediction.ravel())
    return score

def getModel_vConv(config):
    window = config['window'][0]
    model = Sequential()
    pd1 = 'same' if window < 4 else 'valid'
    pd2 = 'same' if window < 5 else 'valid'
    model.add(Conv1D(config['filters1'][0],kernel_size=3, padding=pd1, activation='relu',input_shape=(window, 1)))
    model.add(Conv1D(config['filters2'][0],kernel_size=3, padding=pd2, activation='relu'))
    if config['layers'][0]==3:
        model.add(Conv1D(config['filters3'][0],kernel_size=3, activation='relu'))
    model.add(Dropout(config['dropout'][0]))
    model.add(Flatten())
    model.add(Dense(FORECASTING_STEPS))
    lr = math.pow(10,-config['lr'][0])
    opt = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='mse', optimizer=opt,  metrics=['mean_absolute_error'])
    return model

def evaluateConv(train,test,state,rep):
    scaler = SCALER_MINMAX()
    EPOCHS = 50
    BATCH_SIZE =1
    #config ={'window':[12],'layers':[2],'filters3':[21],'filters2':[31],'filters1':[8],'lr':[2],'dropout':[0]}
    config = {'window':[3],'layers':[2],'filters1':[4],'filters2':[64],'filters3':[2],'dropout':[0],'lr':[2]}
    look_back = config['window'][0]
    
    x_train,y_train = create_dataset_scaled(train,scaler,look_back)
    model = getModel_vConv(config)
    if os.path.isfile('Models/Conv_mm_'+str(state)+'_'+str(rep)+'.h5'):
        model = load_model('Models/Conv_mm_'+str(state)+'_'+str(rep)+'.h5')
    else:
        model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
        model.save('Models/Conv_m_'+str(state)+'_'+str(rep)+'.h5')
    x_test =train[-look_back:]
    y_test = test
    x_test,y_test = normalizeInputTouple(x_test,y_test,scaler)
    x_test = reshapeInputs([x_test])
    y_test = np.array([y_test])
    

    return evaluateModel(model,x_test,y_test,scaler,True)



def getModel_vLSTM(config):
    window = config['window'][0]
    model = Sequential()
    return_sequences = True if config['layers'][0]==3 else False
    model.add(LSTM(config['cs1'][0], activation='tanh',input_shape=(window, 1),return_sequences = True))
    model.add(LSTM(config['cs2'][0], activation='tanh',return_sequences=return_sequences))
    if config['layers'][0]==3:
         model.add(LSTM(config['cs3'][0], activation='tanh'))
    model.add(Dropout(config['dropout'][0]))
    model.add(Dense(FORECASTING_STEPS))
    lr = math.pow(10,-config['lr'][0])
    opt = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='mse', optimizer=opt,  metrics=['mean_absolute_error'])
    return model

def evaluateLSTM(train,test,state,rep):
    scaler = SCALER_MINMAX()
    EPOCHS = 50
    BATCH_SIZE =1
    config = {'window':[3],'layers':[2],'cs1':[2],'cs2':[2],'cs3':[32],'dropout':[0.0],'lr':[1]}
    #config = {'window':[12],'layers':[2],'cs1':[16],'cs2':[8],'cs3':[15],'dropout':[0],'lr':[3]}
    look_back = config['window'][0]
    
    x_train,y_train = create_dataset_scaled(train,scaler,look_back)

    model = getModel_vLSTM(config)
    if os.path.isfile('Models/LSTM_mm_'+str(state)+'_'+str(rep)+'.h5'):
        model = load_model('Models/LSTM_mm_'+str(state)+'_'+str(rep)+'.h5')
    else:
        model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
        model.save('Models/LSTM_mm_'+str(state)+'_'+str(rep)+'.h5')
    x_test =train[-look_back:]
    y_test = test
    x_test,y_test = normalizeInputTouple(x_test,y_test,scaler)
    x_test = reshapeInputs([x_test])
    y_test = np.array([y_test])
    

    return evaluateModel(model,x_test,y_test,scaler,True)

def ensembleModels(models, model_input):
    # collect outputs of models in a list
    yModels=[model(model_input) for model in models]     # averaging outputs
    yAvg=average(yModels)     # build model from same input and avg output
    modelEns = Model(inputs=model_input, outputs=yAvg,    name='ensemble')  
   
    return modelEns

def evaluateLSTM_CONV_Ens(train,test,state,rep):
    scaler = SCALER_MINMAX()
    EPOCHS = 50
    BATCH_SIZE =1
    configLSTM = {'window':[3],'layers':[2],'cs1':[2],'cs2':[2],'cs3':[32],'dropout':[0.0],'lr':[1]}
    configConv = {'window':[3],'layers':[2],'filters1':[4],'filters2':[64],'filters3':[2],'dropout':[0],'lr':[2]}
    look_back = 3#config['window'][0]
    
    x_train,y_train = create_dataset_scaled(train,scaler,look_back)
    modelEns = []
    if os.path.isfile('Models/ens_mm_'+str(state)+'_'+str(rep)+'.h5'):
        modelEns = load_model('Models/ens_mm_'+str(state)+'_'+str(rep)+'.h5')
    else:
        modelL = getModel_vLSTM(configLSTM)
        modelL.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
        modelC = getModel_vConv(configConv)
        modelC.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
        modelL.name = 'vLSTM'
        modelC.name = 'vConv'

        models=[modelL,modelC]
        model_input = Input(shape=models[0].input_shape[1:]) # c*h*w
        modelEns = ensembleModels(models, model_input)
        modelEns.save('Models/ens_mm_'+str(state)+'_'+str(rep)+'.h5')
    x_test =train[-look_back:]
    y_test = test
    x_test,y_test = normalizeInputTouple(x_test,y_test,scaler)
    x_test = reshapeInputs([x_test])
    y_test = np.array([y_test])
    

    return evaluateModel(modelEns,x_test,y_test,scaler,True)

uber_res = []
uber_count = 0
def init_uber(rep):
    global uber_count
    global uber_res
    config = {'window':[15],'noise':[0.0005],'batchsize':[3],'epochs':[50],'lr':[0.006051],'l2':[0.0008],'cs1':[10]}
    uber_res = uber.evaluateModel(config,rep)
    uber_count = 0


def evaluateUber(train,test,state,rep):
    global uber_count
    global uber_res
    #orig :config = {'noise':[0.0005],'batchsize':[3],'epochs':[52],'lr':[0.006051],'l2':[0.0008],'cs1':[10]}
    res = uber_res[uber_count]
    uber_count = uber_count+1
    return res

uber_res2 = []
uber_count2 = 0
def init_uber2(rep):
    global uber_count2
    global uber_res2
    config = {'window':[3],'noise':[0.0005],'batchsize':[1],'epochs':[50],'lr':[2.958487],'l2':[0.0000],'cs1':[64],'cs2':[2]}
    uber_res2 = uber2.evaluateModel(config,rep)
    uber_count2 = 0


def evaluateUber2(train,test,state,rep):
    global uber_count2
    global uber_res2
    res = uber_res2[uber_count2]
    uber_count2 = uber_count2+1
    return res



In [13]:
df = pd.read_csv(DATASET_STATES_CSV,index_col=0,parse_dates=True,encoding = "utf-8")
df = normalize_df(df,DATA_SCALER())


(67, 16)

In [14]:
Final_Results = {}
for rep in range(0,1):
    overall_test_scores = {}
    nbr_of_states = len(list(df))
    #init_uber(rep)
    #display('')
    #init_uber2(rep)
    approaches = []
    approaches.append({'name':'Conv', 'func': evaluateConv})
    approaches.append({'name':'LSTM', 'func': evaluateLSTM})
    #approaches.append({'name':'Uber', 'func': evaluateUber})
    #approaches.append({'name':'Uber_adj', 'func': evaluateUber2})
    approaches.append({'name':'Ens_C_L', 'func': evaluateLSTM_CONV_Ens})
    start = time.time()
    state_nr = 0
    for state in df:
        print(f'{state}:')
        for approach in approaches:
            series = df[state]
            x,y = getTrainTestSplit(series)
            test_score = approach['func'](x,y,state_nr,rep)
            #print(approach['name']+':'+ str(test_score))
            prevStateDict = Final_Results.get(state,{})
            prevRes = prevStateDict.get(approach['name'],[])
            prevRes.append(test_score)
            prevStateDict[approach['name']] = prevRes
            Final_Results[state] = prevStateDict
            print(approach['name']+'_sqrt:'+ str(sqrt(test_score)))
            overall_test_scores[approach['name']]= overall_test_scores.get(approach['name'],0) + test_score
        #print('')
        state_nr = state_nr +1
    for approach in approaches:
        overall_test_scores[approach['name']] = overall_test_scores[approach['name']]/nbr_of_states
        print(approach['name']+'_sqrt:'+str(sqrt(overall_test_scores[approach['name']])))
    end = time.time()
    print(f'{(end - start)/60} minutes runtime')

with open('my_Final_Res_NN.json', 'w') as f:
    json.dump(Final_Results, f)

Schleswig-Holstein:
Conv_sqrt:0.16827581468725505
LSTM_sqrt:0.2052865676396159
Ens_C_L_sqrt:0.1600345362546952
Freie und Hansestadt Hamburg:
Conv_sqrt:0.14808925974056486
LSTM_sqrt:0.12665861547826762
Ens_C_L_sqrt:0.1240626683304279
Niedersachsen:
Conv_sqrt:0.1411971896285601
LSTM_sqrt:0.11484819709668301
Ens_C_L_sqrt:0.12503221585691773
Freie Hansestadt Bremen:
Conv_sqrt:0.2410858711195302
LSTM_sqrt:0.2450555513946636
Ens_C_L_sqrt:0.2273208970809603
Nordrhein-Westfalen:
Conv_sqrt:0.10369753169371367
LSTM_sqrt:0.10672334377296311
Ens_C_L_sqrt:0.10309561671278651
Hessen:
Conv_sqrt:0.14853805628420072
LSTM_sqrt:0.1484532874140613
Ens_C_L_sqrt:0.14479128683738188
Rheinland-Pfalz:
Conv_sqrt:0.15645788738368893
LSTM_sqrt:0.16441988473746577
Ens_C_L_sqrt:0.17625409556610702
Baden-Württemberg:
Conv_sqrt:0.08365999460149424
LSTM_sqrt:0.08026645663642452
Ens_C_L_sqrt:0.08126166760130418
Freistaat Bayern:
Conv_sqrt:0.16249108492380873
LSTM_sqrt:0.1605262303648378
Ens_C_L_sqrt:0.16131378496190638

In [7]:
display(Final_Results)

{'Schleswig-Holstein': {'Conv': [0.04327168172570617,
   0.030380822803602978,
   0.05203594982801513,
   0.03185465157290875,
   0.02972659149219419,
   0.029501900042433915,
   0.02710865086894876,
   0.04781831942368712,
   0.03249226629200718,
   0.02583780043283805],
  'LSTM': [0.03897946526948722,
   0.04003883415172031,
   0.03885520576047641,
   0.03988411412985448,
   0.03741963954890904,
   0.03863279551380397,
   0.038436100565691224,
   0.03842688892613638,
   0.03720484269194127,
   0.03819878546278661],
  'Ens_C_L': [0.03678007745611734,
   0.03244673629054507,
   0.034385331452415256,
   0.039998017086477225,
   0.0325758542447834,
   0.03623899372258237,
   0.034322973116122917,
   0.0350429595514149,
   0.0342189108717041,
   0.03562591976237082]},
 'Freie und Hansestadt Hamburg': {'Conv': [0.021995400176618867,
   0.021038767436020716,
   0.014813619860890768,
   0.016436225317488982,
   0.015422296623473811,
   0.015761522544375735,
   0.021282587102317587,
   0.0142